In [1]:
import pandas as pd
import numpy as np

In [4]:
#import train
train = pd.read_csv('train.csv')
#import test
test = pd.read_csv('test.csv')
#import sample submission
samp = pd.read_csv('sample_submission.csv')

In [5]:
#EDA
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB
